## chapter 12

### question 2

d)

$f(x) = e^{-x}e^{-e^{-x}}$

F(x) = $e^{e^{-x}} = u$

$e^{-x} = -\ln u$

$x = -\ln (-\ln u)$

sample from uniform distribution and $x = -\ln (-\ln u)$

e) 

f(x) = ${1 \over \pi \sqrt{x(1-x)} } 1_{(0,1)}(x)$

$F(x) = {1 \over \pi} \arcsin(\sqrt{x-a \over b-a})= u$

$\pi u = \arcsin(\sqrt{x-a \over b-a})$

$(b-a)\sin^2(\pi u) = x-a$

$(b-a)\sin^2(\pi u)+ a = x$

sample from uniform distribution and $x = (b-a)\sin^2(\pi u)+ a $

f) 

$f(x) = \alpha x^{\alpha -1}$

$F(x) = x^{\alpha} = u$

$x = u^{1 \over \alpha}$



sample from uniform distribution and $x = u^{1 \over \alpha}$

### question 5

we use inverse transfermation, cumulative probabilities are examined in turn until one exceeds lambda.

In [2]:
import Pkg; Pkg.add("StatsBase")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [2913bbd2] + StatsBase v0.33.21
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [33]:
using StatsBase, Statistics

In [28]:
function poisson_dev(lambda::T, n::Int) where T <: Real
x = zeros(n)
    for i = 1:n
        # initional value 
        mm = 0 
        p = exp(-lambda)
        s = p
        u = rand(1)
    while u[1] > s
    mm = mm + 1
    p = p*lambda/mm # probability at y
    s = s + p #  cumulative probabilities
        end
    x[i] = mm  
    end
return x
end

poisson_dev (generic function with 1 method)

In [58]:
outcome = poisson_dev(2,200)

200-element Vector{Float64}:
 1.0
 2.0
 4.0
 2.0
 3.0
 1.0
 0.0
 0.0
 2.0
 1.0
 1.0
 6.0
 2.0
 ⋮
 2.0
 0.0
 1.0
 1.0
 4.0
 2.0
 2.0
 2.0
 0.0
 4.0
 1.0
 2.0

In [59]:
mean(outcome)

1.94

In [60]:
var(outcome)

1.9059296482412063

### question 6

$f(x) = x^{\alpha - 1}(1-x)^{\beta - 1}$

$f'(x) =(1-x)^{\beta - 1}\alpha x^{\alpha - 2} - x^{\alpha - 1} \beta (1-x)^{\beta - 2} = x^{\alpha - 2} (1-x)^{\beta - 2}  [\alpha(1-x) - \beta x] $

let $f'(x_0) = 0$ we get the maximum

$ [\alpha(1-x) - \beta x] = 0$

$x_0 = {\alpha \over \alpha + \beta}$

$f(x_0) = {({\alpha \over \alpha + \beta})^{\alpha - 1}}{({\beta \over \alpha + \beta})^{\beta - 1}} = c$

we implment it in to julia code

In [128]:
function beta_dev(alpha::T, beta::T, n::Int)where T <: Real
i = 1
x = zeros(n)
# calculate c
c = (alpha/(alpha + beta))^(alpha -1)*(beta/(alpha + beta))^(beta-1)
betafunc = SpecialFunctions.beta(alpha,beta)
while i < (n+1)
    u = rand(1)[1] # sample from uniform distribution
    y = rand(1)[1] # sample y from uniform distribution
    p = (1/betafunc) * y^(alpha - 1)*(1-y)^(beta - 1) # calculate p(y)
    cu = u*c
    if cu < p # if u*c < p(y) accept
        x[i] = y
    end
        i = i+1
end
return x
end



beta_dev (generic function with 1 method)

In [206]:
# mean of outcome 2 is 0.5
outcome2 =  beta_dev(2,2,2000)
# mean of outcome 3 is 2/3
outcome3 =  beta_dev(2,4,2000)

2000-element Vector{Float64}:
 0.0
 0.37757520261070077
 0.6832533005672742
 0.4621426997071282
 0.6657293939891524
 0.0
 0.32589025272617544
 0.5833140330112684
 0.06480900285285196
 0.35407629515682937
 0.7685932659641982
 0.0
 0.2364560588790735
 ⋮
 0.5605802033888094
 0.2517255209083028
 0.35896956050479467
 0.0
 0.4367252150831722
 0.29611071439974523
 0.5825318843300517
 0.49049591102805035
 0.6970734916333179
 0.2906006378351037
 0.2898080127652668
 0.0

In [207]:
mean(outcome2)

0.4803393583834385

In [208]:
mean(outcome3)

0.37275254776403527

### 16 Design and implement a Gibbs sampler for generating bivariate normal deviates.

we know the theorem that if $X_1, X_2$ ~ BN$\mu_1, \mu_2, \sigma_1^2, \sigma^2_2, \rho$

$X_1|X2 = x_2$~N$(\mu_1 + \rho {\sigma_1 \over \sigma_2}(x_2-\mu_2), \sigma^2_1(1-\rho^2))$ 

so we have the algorithm

1. set initional value x1 = mu1, x2 = mu2

2. sample x1 from N$(\mu_1 + \rho {\sigma_1 \over \sigma_2}(x_2-\mu_2), \sigma^2_1(1-\rho^2))$ 

3. update x1

4. sample x2 fromN$(\mu_2 + \rho {\sigma_2 \over \sigma_2}(x_1-\mu_1), \sigma^2_2(1-\rho^2))$

5. update x2

we implement the algorithm into julia

In [101]:
using Random, Distributions

In [163]:
function gibb_biv_norm(n::Int, mu1::T,mu2::T, sig1 ::T,sig2 ::T,pho::T )where T <: Real
    # mu1, mu2 are the sample mean, sig1 sig2 are the variance, 
    # pho is covariance parameter
    i = 2
    x = zeros(n, 2)
    x[1,1] = mu1
    x[1,2] = mu2
while i < (n + 1)
        x2 = x[i-1,2]
        m1 = mu1 +(x2 - mu2)*pho*(sig1/sig2)
        d1 = Normal(m1, ((1-pho^2)*sig1)^0.5)
        x[i,1] = rand(d1,1)[1]
        x1 = x[i,1]
        m2 = mu2 + (x1-mu1)*pho*(sig2/sig1)
        d2 = Normal(m2, ((1-pho^2)*sig2)^0.5)
        x[i,2] = rand(d2,1)[1]
        i = i + 1
    end
    return x
end

gibb_biv_norm (generic function with 1 method)

In [182]:
outcme4 =  gibb_biv_norm(5000,1,1,1,1,0.3)

5000×2 Matrix{Float64}:
  1.0         1.0
  0.0685173   1.41434
  2.21586     1.39197
  1.00594     1.70267
  0.209411    2.675
 -0.144399    0.186997
  1.09934     0.41919
 -0.756052    1.69263
  3.13169     2.59985
  1.48165     1.39216
 -0.379695   -0.0127065
  2.34958     0.739501
 -0.0200622   0.358363
  ⋮          
 -0.0364555   1.36146
  1.3605      1.76135
  0.481195    1.44124
  1.02104     0.0062536
  1.25978     0.668518
  3.19971     1.52422
  0.503777    1.52714
  1.52976    -0.388379
  0.630576    0.923322
  0.0548858   2.58413
  0.391726    1.67589
  1.89976     0.938975

In [183]:
# 2000 burn in 
outcome41 = outcme4[2000:4000,1]
outcome42 = outcme4[2000:4000,2]

2001-element Vector{Float64}:
  1.3360148031020096
  0.9950284636716196
  2.322038303719833
  1.674125098488624
  1.2762453576098638
  0.532803864259255
  0.4647711043106298
  1.216845068188944
  2.2099986241523633
  0.9793170718053966
  0.3569379817545506
  0.12824230880811804
  0.3324078290596011
  ⋮
  2.828824091178733
  2.448031864893587
  1.5925482457504967
 -0.23936712983330444
  0.44402973023347225
  0.7037676883571882
  0.6901287983274437
  2.3139884456840183
  1.880552186671209
 -0.05845914415373876
  1.5636889154935707
  0.6227290679921518

In [187]:
cor(outcome41, outcome42)

0.3092233226046545

In [188]:
mean(outcome41)

1.0181769721663672

In [189]:
mean(outcome42)

0.9717803941857359

## chapter11

### question 3

1. $c_j = 1$ 

$\hat{c} = {1\over n} \sum (e^{-2\pi i \over n})^{jk} $

when k = 0, $\hat{c} = 1$

when $k \neq 0$, $\hat{c} = {1\over n} {1-(e^{-2\pi i \over n})^{nk}
\over 1-(e^{-2\pi i \over n})^{k}} = {1\over n}  {1-1
\over 1-(e^{-2\pi i \over n})^{k}} = 0$



2. $j = 0, c_j = 1$

$\hat{c} = {1 \over n} *1 = {1\over n}$

3.

$\hat{c} ={1\over n}(  {1-(e^{-4\pi i \over n})^{nk \over 2}
\over 1-(e^{-4 \pi i \over n})^{k}} + 
 {1-(-e^{-4\pi i \over n})^{nk \over 2}
\over 1-(-e^{-4 \pi i \over n})^{k}})$

if k = 0, $\hat{c} = 0$

if $k \neq 0$, $\hat{c} = 0$ if ${n \over 2}$ is even

$\hat{c} ={1\over n} { 1\over 1-(-e^{-4 \pi i \over n})^{k}}$ if  ${n \over 2}$ is odd

4. 
$c_j = 1$  for $j = 0, 1, {1 \over 2} - 1$

$\hat{c} = {1\over n} \sum^{{1 \over 2} - 1} (e^{-2\pi i \over n})^{jk} $

when k = 0, $\hat{c} = {1 \over 2}$

when $k \neq 0$, $\hat{c} = {1\over n} {1-(e^{-2\pi i \over n})^{nk \over 2}
\over 1-(e^{-2\pi i \over n})^{k}} = 
{1\over n} {1-(e^{-\pi i k})
\over 1-(e^{-2\pi i \over n})^{k}}$

if k is even, then $\hat{c} = 0$

if k is odd, then $\hat{c} ={1\over n} {2
\over 1-(e^{-2\pi i \over n})^{k}} $

### question 6

$\hat{T_r c_k} = {1 \over rn} \sum^{rn-1}S_r c_j u^{-jk}$
 
$ = {1\over rn} \sum^{rn-1} c_{j-r} u^{-jk} $
 
$ = {1 \over n} \sum^{n-1}u^{-jk}$
 
$ = {1 \over n} u^{-jk} \sum^{n-1}c_{j-r} u^{-(j-r)k}$

$= {1 \over n} u^{-jk} \sum^{n-1}c_{l} u^{-(l)k}$
 
$ = u^{-jk} \hat{c}$

$\hat{Rc_k} = {1\over n}\sum Rc_j u^{-kj}$

$= R{1\over }\sum Rc_j u^{-kj}$

$ = R \hat{c}$